In [22]:
import os

STORY_DIR = "story"


def load_all_stories():
    """
    Read all story files and return:
      - stories[file] = {"from_file": str|None, "title": str, "text": str}
      - links[file] = list of child files (filenames)
    Resolution rules for From:
      - case-insensitive
      - accepts "start" or "start.txt"
      - "none" (or empty) -> treated as no parent
      - if From cannot be resolved to an existing filename, it is ignored
    """
    # collect filenames
    files = sorted(
        [f for f in os.listdir(STORY_DIR) if f.lower().endswith(".txt")]
    )

    # map lowercased filename -> real filename (for case-insensitive resolution)
    lower_to_real = {f.lower(): f for f in files}

    stories = {}
    links = {}

    for filename in files:
        path = os.path.join(STORY_DIR, filename)
        with open(path, "r", encoding="utf-8") as fh:
            raw_lines = [ln.rstrip("\n") for ln in fh.readlines()]

        # parse fields (allow empty lines between header and body)
        from_raw = None
        title = None
        text_lines = []
        for ln in raw_lines:
            stripped = ln.strip()
            if not stripped and text_lines:
                # empty line after we started text -> preserve paragraph break
                text_lines.append("")
                continue

            low = stripped.lower()
            if low.startswith("from:"):
                # raw parent text (keep original casing for better error messages)
                from_raw = stripped.split(":", 1)[1].strip()
            elif low.startswith("title:"):
                title = stripped.split(":", 1)[1].strip()
            else:
                text_lines.append(ln)

        # normalize/resolve the parent (from_raw) to one of the actual filenames, if possible
        parent = None
        if from_raw:
            r = from_raw.strip().lower()
            if r not in ("", "none"):
                # try directly (maybe contributor wrote "start.txt")
                if r in lower_to_real:
                    parent = lower_to_real[r]
                else:
                    # try with ".txt" appended (contributor wrote "start")
                    if not r.endswith(".txt") and (r + ".txt") in lower_to_real:
                        parent = lower_to_real[r + ".txt"]
                    else:
                        # try matching against filename without extension (e.g. "start" -> "start.txt")
                        for lower_fname, real_fname in lower_to_real.items():
                            if lower_fname.endswith(".txt") and lower_fname[:-4] == r:
                                parent = real_fname
                                break
                # if parent still None, we ignore the bad From: value to avoid orphan links
                # (we preserve the raw value in stories for debugging if desired)

        stories[filename] = {
            "from_file": parent,               # resolved real filename or None
            "title": title if title else filename,
            "text": "\n".join(text_lines).strip()
        }

        # only create a link if parent is a real, resolved filename
        if parent:
            links.setdefault(parent, []).append(filename)

    return stories, links


In [23]:

# Debug print
stories, links = load_all_stories()
print(f"{stories = }")
print('*' * 40)
print(f"{links = }")


stories = {'abandoned_hut.txt': {'from_file': None, 'title': 'abandon hut', 'text': 'Hidden between mossy oaks, you spot an abandoned wooden hut. \nThe door creaks open as the wind whistles through the cracks.'}, 'cave.txt': {'from_file': 'start.txt', 'title': 'Explore the Cave', 'text': 'The cave is cold and damp. You see glowing eyes\nstaring at you in the darkness.'}, 'riverbank.txt': {'from_file': None, 'title': 'river bank', 'text': 'You push through the trees and find a sparkling river. \nThe water rushes swiftly, and you see a narrow path following the riverbank.'}, 'start.txt': {'from_file': None, 'title': 'The Beginning', 'text': 'You are standing at the entrance of a dark forest.\nThe trees look tall and ancient.'}}
****************************************
links = {'start.txt': ['cave.txt']}


In [24]:
# show all from_files in stories
links

{'start.txt': ['cave.txt']}